<a href="https://colab.research.google.com/github/haoyuliao/ml-code-examples-HYL/blob/main/detect_ai_vs_human_generated_images_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import time
import copy
import random
import numpy as np
import pandas as pd
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader, Subset, random_split
import torchvision.transforms as transforms
import torchvision.transforms.v2 as T
from torchvision.models import convnext_base
from torchvision.models.vision_transformer import vit_b_16, vit_l_16
from PIL import Image, ImageFilter
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import shutil
